In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dredFISH.Utils import basicu
from dredFISH.Utils import powerplots
from dredFISH.Visualization import compile_tex

import importlib
importlib.reload(powerplots)
importlib.reload(compile_tex)

# 12-section brain dataset

In [ ]:
path_dataset = '/bigstore/GeneralStorage/Data/dredFISH/Dataset3-t1'
path_fig = os.path.join(path_dataset, 'figures')
if not os.path.isdir(path_fig):
    os.mkdir(path_fig)

files_mtx = np.sort(glob.glob('*_matrix.csv', dir_fd=path_dataset))
files_meta = np.sort(glob.glob('*_metadata.csv', dir_fd=path_dataset))
files_mtx.shape, files_meta.shape

In [ ]:
sections = {i: f.replace('_matrix.csv', '').replace('DPNMF_1A_2B_11A_12B_2022Jul28_Section_', '')
            for i, f in enumerate(files_mtx)}
sections

# remove non-cells by setting `cytoplasm_size`

In [ ]:
sctn = 0
sctn_name = sections[sctn]
file_mtx = files_mtx[sctn]
file_meta = files_meta[sctn]
print(files_mtx[0], files_meta[0])

mtx = pd.read_csv(os.path.join(path_dataset, file_mtx), sep=',', index_col=0)
mtx.columns = np.char.add('br', np.arange(24).astype(str))
meta = pd.read_csv(os.path.join(path_dataset, file_meta), sep=',', index_col=0)
df = meta.join(mtx)
print(df.shape)

# cond = df['cytoplasm_size'] > 10
cond = df['nuclei_signal'] > 1500
df = df.loc[cond]
mtx = df[np.char.add('br', np.arange(24).astype(str))]
print(df.shape)



In [ ]:
# norm
ftrs_mat = basicu.normalize_fishdata(mtx.values, norm_cell=True, norm_basis=True)
for i in range(ftrs_mat.shape[1]):
    df[f'b{i}'] = ftrs_mat[:,i]
    
# XY
XY = df[['stage_x', 'stage_y']].values
x = XY[:,0] 
y = XY[:,1] 

In [ ]:
output = os.path.join(path_fig, f"fig1_xy_sect{sctn}_{sctn_name}.pdf")
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(x, y, s=1, edgecolor='none', color='black', rasterized=True)
ax.set_title(sctn_name)
ax.set_aspect('equal')
powerplots.savefig_autodate(fig, output)
plt.show()

# output = os.path.join(path_fig, f"fig2_basis_box_sect{sctn}_{sctn_name}.pdf")
# plot_basis_box(ftrs_mat, output=output)

# output = os.path.join(path_fig, f"fig3_basis_xy_sect{sctn}_{sctn_name}.pdf")
# powerplots.plot_basis_spatial(df, xcol='stage_x', ycol='stage_y', vmin=-1, vmax=1, output=output)

# set spatial mask

In [ ]:
from matplotlib.collections import LineCollection

In [ ]:
def draw_control_points(points, ax):
    """
    """
    line_segs = [
        [points[i], points[(i+1)%len(points)]]
        for i in range(len(points))
    ]

    pm = np.asarray(points)
    ax.scatter(pm[:,0], pm[:,1], color='r')
    for i, p in enumerate(points):
        ax.text(p[0], p[1], i)
    lc = LineCollection(line_segs, linewidth=1, colors='r')
    ax.add_collection(lc)
    return line_segs

In [ ]:
# mask = np.zeros()
# mask
points = [
    (-6500, 20400), 
    (-1000, 20600),
    (0, 26000),
    (-1000, 29000),
    (-10000, 29000),
    (-10000, 24000),
]

output = "" #os.path.join(path_fig, f"fig1_xy_sect{sctn}_{sctn_name}.pdf")
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(x, y, s=1, edgecolor='none', color='gray', rasterized=True)
draw_control_points(points,ax)
ax.set_title(sctn_name)
ax.set_aspect('equal')
plt.show()

In [ ]:
def is_in_polygon_proto(poly, p):
    """
    test if p is in the defined polygon
    """
    is_in = False
    poly = np.asarray(poly) 
    npoly = len(poly)
    
    for i in range(npoly):
        j = (i-1) % npoly
        if (((poly[i,1]>p[1]) != (poly[j,1]>p[1])) and
            (p[0] < poly[i,0] + (p[1]-poly[i,1])*(poly[j,0]-poly[i,0])/(poly[j,1]-poly[i,1]))
           ):
            is_in = (not is_in) 
    return is_in

In [ ]:
def is_in_polygon(poly, ps):
    """
    test if each point is in the defined polygon
    """
    # is_in = False
    poly = np.asarray(poly) 
    npoly = len(poly)
    
    ps = np.asarray(ps)
    is_in = np.zeros(len(ps), dtype=int)
    
    for i in range(npoly):
        j = (i-1) % npoly
        if poly[j,1]-poly[i,1] != 0: # not cross
            cond1 = ((poly[i,1]>ps[:,1]) != (poly[j,1]>ps[:,1]))
            cond2 = (ps[:,0] < poly[i,0] + (ps[:,1]-poly[i,1])*(poly[j,0]-poly[i,0])/(poly[j,1]-poly[i,1]))
            cond = np.logical_and(cond1, cond2)
            is_in += cond  
    return is_in % 2

In [ ]:
%%time
res = is_in_polygon(points, XY) #[:100])
res

In [ ]:
output = "" #os.path.join(path_fig, f"fig1_xy_sect{sctn}_{sctn_name}.pdf")
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(XY[:,0], XY[:,1], c=res, s=1, edgecolor='none', rasterized=True, cmap='rocket_r')
draw_control_points(points,ax)
ax.set_title(sctn_name)
ax.set_aspect('equal')
plt.show()